# Práctica 2: Clasificación y evaluación de modelos
Duración: 1 sesión

Apartado 2.3: Clasificación usando agrupaciones de clasificadores

Objetivo: El objetivo de esta práctica es introducir los conceptos de clasificación usando agrupaciones (ensembles) de clasificadores. Usaremos dos clasificadores diferentes para ver cómo se comportan como métodos base de un ensemble.

Realice los siguientes ejercicios usando el módulo scikit-learn de Python y cualquier otro módulo adicional que considere utilizando los mismo conjuntos de datos de la práctica 3:
1. Seleccione como métodos base un árbol de decisión y una máquina de vectores soporte.
2. Para cada uno de estos dos métodos de clasificación realice los siguientes pasos usando validación cruzada de 10
particiones:
 1. Aplique el método base a cada uno de los conjuntos y anote los resultados obtenidos.
 2. Aplique el método de combinación de clasificadores Bagging a cada uno de los conjuntos y anote los
resultados obtenidos.
 3. Seleccione dos algoritmos de Boosting y aplique estos algoritmos a cada uno de los conjuntos y anote los
resultados obtenidos.
 4. Compare si hay diferencias significativas entre ellos usando el test de Iman-Davenport. Si es así, aplique el
procedimiento de Wilcoxon para comparar cada método de agrupación con el clasificador base.
3. Enuncie las conclusiones del estudio indicando la influencia del clasificador base en el rendimiento de las
agrupaciones de clasificadores.
4. Evalúe la influencia de la validación de híper-parámetros en el rendimiento de los ensembles con respecto a
los métodos base.

In [6]:
pip install sklearn-evaluation

In [7]:
#Disable warning of Ripper implementation
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn import tree, svm
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn_evaluation import plot
import seaborn as sns
import time

In [8]:
data = arff.loadarff('ionosphere.arff')
df_iono = pd.DataFrame(data[0])

data = arff.loadarff('diabetes.arff')
df_diabe = pd.DataFrame(data[0])

data = arff.loadarff('vehicle.arff')
df_Vehicle = pd.DataFrame(data[0])

data = arff.loadarff('vowel.arff')
df_vowel = pd.DataFrame(data[0])

data = arff.loadarff('iris.arff')
df_iris = pd.DataFrame(data[0])

data = arff.loadarff('letter.arff')
df_letter = pd.DataFrame(data[0])

In [9]:
# Changing the last categorical class value into a numerical value
df_iono['class'] = pd.factorize(df_iono['class'])[0]

# Changing the last categorical class value into a numerical value
df_diabe['class'] = pd.factorize(df_diabe['class'])[0]

# Changing the last categorical class value into a numerical value
df_Vehicle['Class'] = pd.factorize(df_Vehicle['Class'])[0]

# Changing the last categorical class value into a numerical value
df_vowel['Class'] = pd.factorize(df_vowel['Class'])[0]

# Changing the last categorical class value into a numerical value
df_iris['class'] = pd.factorize(df_iris['class'])[0]

# Changing the last categorical class value into a numerical value
df_letter['class'] = pd.factorize(df_letter['class'])[0]


In [10]:
trainIono, testIono = train_test_split(df_iono, test_size=.25)
X_trainIono = trainIono.drop('class', axis=1)
y_trainIono = trainIono['class']
X_testIono = testIono.drop('class', axis=1)
y_testIono = testIono['class']

trainDiabe, testDiabe = train_test_split(df_diabe, test_size=.25)
X_trainDiabe = trainDiabe.drop('class', axis=1)
y_trainDiabe = trainDiabe['class']
X_testDiabe = testDiabe.drop('class', axis=1)
y_testDiabe = testDiabe['class']

trainVehicle, testVehicle = train_test_split(df_Vehicle, test_size=.25)
X_trainVehicle = trainVehicle.drop('Class', axis=1)
y_trainVehicle = trainVehicle['Class']
X_testVehicle = testVehicle.drop('Class', axis=1)
y_testVehicle = testVehicle['Class']

trainVowel, testVowel = train_test_split(df_vowel, test_size=.25)
X_trainVowel = trainVowel.drop('Class', axis=1)
y_trainVowel = trainVowel['Class']
X_testVowel = testVowel.drop('Class', axis=1)
y_testVowel = testVowel['Class']

trainIris, testIris = train_test_split(df_iris, test_size=.25)
X_trainIris = trainIris.drop('class', axis=1)
y_trainIris = trainIris['class']
X_testIris = testIris.drop('class', axis=1)
y_testIris = testIris['class']

trainLetter, testLetter = train_test_split(df_letter, test_size=.25)
X_trainLetter = trainLetter.drop('class', axis=1)
y_trainLetter = trainLetter['class']
X_testLetter = testLetter.drop('class', axis=1)
y_testLetter = testLetter['class']

In [11]:
data = [
    ('Ionosphere', X_trainIono, y_trainIono, X_testIono, y_testIono),
    ('Diabetes', X_trainDiabe, y_trainDiabe,X_testDiabe, y_testDiabe),
    ('Vehicle', X_trainVehicle, y_trainVehicle, X_testVehicle, y_testVehicle),
    ('Vowel', X_trainVowel, y_trainVowel, X_testVowel, y_testVowel),
    ('Iris', X_trainIris, y_trainIris, X_testIris, y_testIris)
]

# Comparison without using GridSearch

In [12]:
clf_tree = tree.DecisionTreeClassifier()
clf_svm = svm.SVC()
clf_BaggingTree = BaggingClassifier(estimator=clf_tree)
clf_AdaBoostSAMMETree = AdaBoostClassifier(estimator=clf_tree,algorithm='SAMME')
clf_AdaBoostSAMMERTree = AdaBoostClassifier(estimator=clf_tree,algorithm='SAMME.R')
clf_BaggingSVM = BaggingClassifier(estimator=clf_svm)
clf_AdaBoostSAMMESVM = AdaBoostClassifier(estimator=clf_svm,algorithm='SAMME')
clf_AdaBoostSAMMERSVM = AdaBoostClassifier(estimator=clf_svm,algorithm='SAMME.R')
clf_GradBoost = GradientBoostingClassifier()

In [13]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreSVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_svm,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreSVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.9354700854700855 +- 0.041320609835001056
Tiempo medio en ejecutarse el método (train): 0.011040067672729493 +- 0.011040067672729493s
Tiempo medio en ejecutarse el método (score): 0.0065200567245483395 +- 0.0065200567245483395s
Tiempo en ejecutarse la búsqueda 2.694411039352417s, (0.04490685065587362 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7673018753781004 +- 0.02396020444414765
Tiempo medio en ejecutarse el método (train): 0.020153498649597167 +- 0.020153498649597167s
Tiempo medio en ejecutarse el método (score): 0.006453418731689453 +- 0.006453418731689453s
Tiempo en ejecutarse la búsqueda 0.16493487358093262s, (0.00274891455968221 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.774454365079365 +- 0.029626676912587448
Tiempo medio en ejecutarse el método (train): 0.023032498359680176 +- 0.023032498359680176s
Tiempo medio en ejecutarse el m

In [14]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreTree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_tree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreTree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreTree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8477207977207977 +- 0.06365290213770579
Tiempo medio en ejecutarse el método (train): 0.011526870727539062 +- 0.011526870727539062s
Tiempo medio en ejecutarse el método (score): 0.004701423645019531 +- 0.004701423645019531s
Tiempo en ejecutarse la búsqueda 0.13386106491088867s, (0.0022310177485148114 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7189655172413792 +- 0.059408008699639664
Tiempo medio en ejecutarse el método (train): 0.008754444122314454 +- 0.008754444122314454s
Tiempo medio en ejecutarse el método (score): 0.003648829460144043 +- 0.003648829460144043s
Tiempo en ejecutarse la búsqueda 0.09942007064819336s, (0.0016570011774698892 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9179811507936508 +- 0.01972094280306819
Tiempo medio en ejecutarse el método (train): 0.00759437084197998 +- 0.00759437084197998s
Tiempo medio en ejecutarse el

In [15]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBaggingSVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_BaggingSVM,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBaggingSVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBaggingSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.9316239316239316 +- 0.04049694954010985
Tiempo medio en ejecutarse el método (train): 0.07846007347106934 +- 0.07846007347106934s
Tiempo medio en ejecutarse el método (score): 0.0173443078994751 +- 0.0173443078994751s
Tiempo en ejecutarse la búsqueda 0.555586576461792s, (0.0092597762743632 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7637326073805203 +- 0.03916681240522346
Tiempo medio en ejecutarse el método (train): 0.11821725368499755 +- 0.11821725368499755s
Tiempo medio en ejecutarse el método (score): 0.020959067344665527 +- 0.020959067344665527s
Tiempo en ejecutarse la búsqueda 0.7479491233825684s, (0.012465818723042806 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.7681795634920634 +- 0.02179551582301958
Tiempo medio en ejecutarse el método (train): 0.11349108219146728 +- 0.11349108219146728s
Tiempo medio en ejecutarse el método (score):

In [16]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBaggingTree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_BaggingTree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBaggingTree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBaggingTree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8934472934472936 +- 0.04469757704713748
Tiempo medio en ejecutarse el método (train): 0.11371309757232666 +- 0.11371309757232666s
Tiempo medio en ejecutarse el método (score): 0.009350419044494629 +- 0.009350419044494629s
Tiempo en ejecutarse la búsqueda 0.7013664245605469s, (0.011689440409342448 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7290986085904416 +- 0.049147647336200355
Tiempo medio en ejecutarse el método (train): 0.22492485046386718 +- 0.22492485046386718s
Tiempo medio en ejecutarse el método (score): 0.023783040046691895 +- 0.023783040046691895s
Tiempo en ejecutarse la búsqueda 1.3449058532714844s, (0.02241509755452474 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.941641865079365 +- 0.015823354387363323
Tiempo medio en ejecutarse el método (train): 0.20932316780090332 +- 0.20932316780090332s
Tiempo medio en ejecutarse el método (

In [17]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting1_SVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMESVM,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting1_SVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting1_SVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.6427350427350428 +- 0.01415641965675765
Tiempo medio en ejecutarse el método (train): 1.579045820236206 +- 1.579045820236206s
Tiempo medio en ejecutarse el método (score): 0.11202099323272705 +- 0.11202099323272705s
Tiempo en ejecutarse la búsqueda 8.724980115890503s, (0.1454163352648417 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.6614942528735632 +- 0.006527480857241695
Tiempo medio en ejecutarse el método (train): 1.853383684158325 +- 1.853383684158325s
Tiempo medio en ejecutarse el método (score): 0.12757401466369628 +- 0.12757401466369628s
Tiempo en ejecutarse la búsqueda 11.37070918083191s, (0.18951181968053182 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.7649801587301587 +- 0.004612295117503322
Tiempo medio en ejecutarse el método (train): 0.6916273832321167 +- 0.6916273832321167s
Tiempo medio en ejecutarse el método (score): 0.0394985

In [18]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting1_Tree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMETree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting1_Tree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting1_Tree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8371794871794871 +- 0.07039703867375144
Tiempo medio en ejecutarse el método (train): 0.015830302238464357 +- 0.015830302238464357s
Tiempo medio en ejecutarse el método (score): 0.00475924015045166 +- 0.00475924015045166s
Tiempo en ejecutarse la búsqueda 0.13798165321350098s, (0.0022996942202250163 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7310647307924986 +- 0.049429850267089084
Tiempo medio en ejecutarse el método (train): 0.011345744132995605 +- 0.011345744132995605s
Tiempo medio en ejecutarse el método (score): 0.0046727657318115234 +- 0.0046727657318115234s
Tiempo en ejecutarse la búsqueda 0.1191854476928711s, (0.001986424128214518 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9211061507936508 +- 0.021341027288853148
Tiempo medio en ejecutarse el método (train): 0.010860872268676759 +- 0.010860872268676759s
Tiempo medio en ejecutarse e

AdaBoost with SVM and SAMMER_R is not supported

In [19]:
"""
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting2_SVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMERSVM,i[1], y=i[2],cv=10, n_jobs=-1, error_score='raise')
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting2_SVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=Score)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())
"""

'\nDatasets = []\nTimeTrain = []\nTimeScore = []\nTimeCV = []\nScoreBoosting2_SVM = []\nfor i in data:\n    init = time.time()\n    cv = cross_validate(clf_AdaBoostSAMMERSVM,i[1], y=i[2],cv=10, n_jobs=-1, error_score=\'raise\')\n    end = time.time()\n    timeCV = end - init\n    print(f"\n--------- {i[0]} ---------")\n    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv[\'test_score\'])} +- {np.std(cv[\'test_score\'])}")\n    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv[\'fit_time\'])} +- {np.mean(np.mean(cv[\'fit_time\']))}s")\n    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv[\'score_time\'])} +- {np.mean(np.mean(cv[\'score_time\']))}s")\n    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")\n    Datasets.append(i[0])\n    TimeTrain.append(np.mean(cv[\'fit_time\']))\n    TimeScore.append(np.mean(cv[\'score_time\']))\n    TimeCV.append(timeCV)\n    ScoreBoosting2_SVM.append(np.mean(cv[\'test_score\']))\n\nm

In [20]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting2_Tree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMERTree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting2_Tree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting2_Tree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8254985754985755 +- 0.06478663173621244
Tiempo medio en ejecutarse el método (train): 0.014299845695495606 +- 0.014299845695495606s
Tiempo medio en ejecutarse el método (score): 0.005156207084655762 +- 0.005156207084655762s
Tiempo en ejecutarse la búsqueda 0.12090253829956055s, (0.002015042304992676 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.722292800967937 +- 0.04596973746069332
Tiempo medio en ejecutarse el método (train): 0.009687232971191406 +- 0.009687232971191406s
Tiempo medio en ejecutarse el método (score): 0.0034607887268066407 +- 0.0034607887268066407s
Tiempo en ejecutarse la búsqueda 0.09449315071105957s, (0.0015748858451843262 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.925843253968254 +- 0.01743817928244871
Tiempo medio en ejecutarse el método (train): 0.011609244346618652 +- 0.011609244346618652s
Tiempo medio en ejecutarse el

In [21]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreGradBoost = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_GradBoost,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreGradBoost.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreGradBoost)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.9086894586894587 +- 0.0392265896430168
Tiempo medio en ejecutarse el método (train): 0.3920912265777588 +- 0.3920912265777588s
Tiempo medio en ejecutarse el método (score): 0.004993057250976563 +- 0.004993057250976563s
Tiempo en ejecutarse la búsqueda 2.0352988243103027s, (0.03392164707183838 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7551421657592257 +- 0.04092040285321942
Tiempo medio en ejecutarse el método (train): 0.23976457118988037 +- 0.23976457118988037s
Tiempo medio en ejecutarse el método (score): 0.004790067672729492 +- 0.004790067672729492s
Tiempo en ejecutarse la búsqueda 1.2541770935058594s, (0.02090295155843099 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9557787698412697 +- 0.028276430973572984
Tiempo medio en ejecutarse el método (train): 0.5351858377456665 +- 0.5351858377456665s
Tiempo medio en ejecutarse el método (score)

In [22]:
compareDF = pd.DataFrame()
compareDF['SVM'] = ScoreSVM
compareDF['Tree'] = ScoreTree
compareDF['BaggingSVM'] = ScoreBaggingSVM
compareDF['BaggingTree'] = ScoreBaggingTree
compareDF['Boosting1_SVM'] = ScoreBoosting1_SVM
compareDF['Boosting1_Tree'] = ScoreBoosting1_Tree
compareDF['Boosting2_Tree'] = ScoreBoosting2_Tree
compareDF['GradBoost'] = ScoreGradBoost
compareDF.to_csv("performance.csv", index=False)
print(compareDF)
print(compareDF.to_latex())

        SVM      Tree  BaggingSVM  BaggingTree  Boosting1_SVM  Boosting1_Tree  \
0  0.935470  0.847721    0.931624     0.893447       0.642735        0.837179   
1  0.767302  0.718966    0.763733     0.729099       0.661494        0.731065   
2  0.774454  0.917981    0.768180     0.941642       0.764980        0.921106   
3  0.964919  0.981117    0.970324     0.987856       0.905532        0.981117   
4  0.981818  0.946212    0.972727     0.946970       0.348485        0.955303   

   Boosting2_Tree  GradBoost  
0        0.825499   0.908689  
1        0.722293   0.755142  
2        0.925843   0.955779  
3        0.979748   0.987874  
4        0.955303   0.973485  
\begin{tabular}{lrrrrrrrr}
\toprule
{} &       SVM &      Tree &  BaggingSVM &  BaggingTree &  Boosting1\_SVM &  Boosting1\_Tree &  Boosting2\_Tree &  GradBoost \\
\midrule
0 &  0.935470 &  0.847721 &    0.931624 &     0.893447 &       0.642735 &        0.837179 &        0.825499 &   0.908689 \\
1 &  0.767302 &  0.718966 &   

In [23]:
import numpy as np
from scipy.stats import chi2

def compute_iman_davenport_statistic(performance_matrix):
  # Compute the ranks of the model performance on each dataset
  ranks = np.apply_along_axis(lambda x: len(x) - np.argsort(np.argsort(x)), 1, performance_matrix)

  # Sum the ranks for each model across all datasets
  rank_sums = np.sum(ranks, axis=0)

  # Compute the iman Davenport statistic
  iman_davenport_statistic = (np.max(rank_sums) - np.min(rank_sums)) / performance_matrix.shape[1]

  return iman_davenport_statistic

def compute_p_value(iman_davenport_statistic, num_models, num_datasets):
  # Compute the degrees of freedom for the iman Davenport test
  df = num_models - 1

  # Compute the p-value using the chi-squared distribution
  p_value = 1 - chi2.cdf(iman_davenport_statistic, df)

  return p_value

def iman_davenport_test(performance_matrix, significance_level):
  # Compute the iman Davenport statistic and p-value
  iman_davenport_statistic = compute_iman_davenport_statistic(performance_matrix)
  p_value = compute_p_value(iman_davenport_statistic, performance_matrix.shape[0], performance_matrix.shape[1])

  # Determine whether the difference in performance between the models is statistically significant
  if p_value < significance_level:
    print(f"The difference in performance between the models is statistically significant (p = {p_value:.3f})")
  else:
    print(f"The difference in performance between the models is not statistically significant (p = {p_value:.3f})")
  return p_value

In [24]:
performance_matrix = pd.read_csv("performance.csv")
# Run the iman Davenport test
p_value = iman_davenport_test(performance_matrix, 0.05)

The difference in performance between the models is not statistically significant (p = 0.441)


In [25]:
# If there are significant differences between the models, then apply the wilcoxon test to determine which models are significantly different
if p_value < 0.05:
  # Compute the pairwise differences between the models
  pairwise_differences = np.apply_along_axis(lambda x: x - x[:, None], 1, performance_matrix)

  # Compute the p-values for the pairwise differences using the Wilcoxon signed-rank test
  p_values = np.apply_along_axis(lambda x: wilcoxon(x, zero_method="wilcox")[1], 1, pairwise_differences)

  # Compute the Bonferroni correction
  bonferroni_correction = 0.05 / (p_values.shape[0] * (p_values.shape[0] - 1) / 2)

  # Determine which models are significantly different
  significant_differences = np.argwhere(p_values < bonferroni_correction)

  # Print the significant differences
  for difference in significant_differences:
    print(f"Model {difference[0]} is significantly different from model {difference[1]}")

# Comparison using GridSearch

In [26]:
parametersSVM = [
    {"kernel": ["linear"], "C": [1, 10]}
]
parametersTree = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter' : ['best', 'random']
}
parametersSVMBagging = [
    {
    'n_estimators':[10], # Numero de estimators = 10 porque si no tarda demasiado
    "estimator__kernel": ["rbf"],
    "estimator__C": [0.01, 0.1],
    'estimator__gamma': [0.01, 0.1],
    #'max_samples': [0.75, 1]
    # Saltan warnings si aleatoriamente solo seleccionamos instancias de una clase. Podríamos ignorar dichos fits o capturar los warning. Eliminamos el problema directamente
    'max_features': [0.5, 0.75],
    'bootstrap': [True, False]
    }
]
parametersTreeBagging = {
    'estimator__criterion':['gini', 'entropy', 'log_loss'],
    'estimator__splitter' : ['best', 'random'],
    # {'max_samples': [0.75, 1],
    'max_features': [0.5, 0.75, 1],
    'bootstrap': [True, False]
}
parametersSVMBoosting = [
    {
    "estimator__kernel": ["rbf"],
    "estimator__C": [1, 10, 100, 1000],
    'estimator__gamma': [0.01, 0.1, 1],
    'n_estimators':[25, 50],
    'learning_rate':[0.1, 1, 10]
    }
]
parametersTreeBoosting = {
    'estimator__criterion':['gini', 'entropy', 'log_loss'],
    'estimator__splitter' : ['best', 'random'],
    'n_estimators':[25, 50],
    'learning_rate':[0.1,1,10]
}
parametersGradBoosting = {
    "loss" : ['log_loss', 'deviance', 'exponential'],
    'learning_rate' : [0.01, 0.1, 1],
    'n_estimators': [100],
    'criterion': ['friedman_mse', 'squared_error'],
    'warm_start': [True, False]
}

optimalSVM = GridSearchCV(estimator=clf_svm, cv=10, param_grid=parametersSVM, n_jobs=-1)
optimalTree = GridSearchCV(estimator=clf_tree, cv=10, param_grid=parametersTree, n_jobs=-1)
optimalSVMBag = GridSearchCV(estimator=clf_BaggingSVM, cv=10, param_grid=parametersSVMBagging, n_jobs=-1)
optimalTreeBag = GridSearchCV(estimator=clf_BaggingTree, cv=10, param_grid=parametersTreeBagging, n_jobs=-1)
optimalSVMBoost1 = GridSearchCV(estimator=clf_AdaBoostSAMMESVM, cv=10, param_grid=parametersSVMBoosting, n_jobs=-1)
optimalTreeBoost1 = GridSearchCV(estimator=clf_AdaBoostSAMMETree, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalSVMBoost2 = GridSearchCV(estimator=clf_AdaBoostSAMMERSVM, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalTreeBoost2 = GridSearchCV(estimator=clf_AdaBoostSAMMERTree, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalGradBoost = GridSearchCV(estimator=clf_GradBoost, cv=10, param_grid=parametersGradBoosting, n_jobs=-1)

In [27]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreSVM = []
for i in data:
    init = time.time()
    optimalSVM.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVM:")
    print(f'    Best params -> {optimalSVM.best_params_}')
    print(f'    Best score -> {optimalSVM.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVM.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVM.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVM.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVM.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreSVM.append(optimalSVM.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 10, 'kernel': 'linear'}
    Best score -> 0.8783475783475783
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.875
Tiempo medio en ejecutarse el método: 0.013677060604095459 +- 0.0037599614108274806s
Tiempo en ejecutarse la búsqueda 0.2499380111694336s, (0.00416563351949056 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 10, 'kernel': 'linear'}
    Best score -> 0.7776164549304296
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.75
Tiempo medio en ejecutarse el método: 21.518423187732697 +- 5.785629552233049s
Tiempo en ejecutarse la búsqueda 247.28827834129333s, (4.121471305688222 min)

--------- Vehicle ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 10, 'kernel': 'linear'}
    Best score -> 0.963640873015873
Si usamos el data

In [28]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreTree = []
for i in data:
    init = time.time()
    optimalTree.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente Tree:")
    print(f'    Best params -> {optimalTree.best_params_}')
    print(f'    Best score -> {optimalTree.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTree.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTree.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTree.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTree.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreTree.append(optimalTree.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreTree)
TreeDF = pd.DataFrame (my_dict)
print(TreeDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterion': 'entropy', 'splitter': 'random'}
    Best score -> 0.9085470085470085
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9318181818181818
Tiempo medio en ejecutarse el método: 0.013209176063537597 +- 0.004366503370763392s
Tiempo en ejecutarse la búsqueda 0.6956453323364258s, (0.011594088872273763 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterion': 'gini', 'splitter': 'best'}
    Best score -> 0.7241681790683606
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.65625
Tiempo medio en ejecutarse el método: 0.005953383445739746 +- 0.0012378103965433468s
Tiempo en ejecutarse la búsqueda 0.3425593376159668s, (0.005709322293599447 min)

--------- Vehicle ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterion': 'log_loss', 

In [29]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBaggingSVM = []
for i in data:
    init = time.time()
    optimalSVMBag.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBag:")
    print(f'    Best params -> {optimalSVMBag.best_params_}')
    print(f'    Best score -> {optimalSVMBag.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBag.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBag.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBag.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBag.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBaggingSVM.append(optimalSVMBag.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBaggingSVM)
SVMBagDF = pd.DataFrame (my_dict)
print(SVMBagDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVMBag:
    Best params -> {'bootstrap': True, 'estimator__C': 0.1, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'max_features': 0.75, 'n_estimators': 10}
    Best score -> 0.8709401709401708
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.8409090909090909
Tiempo medio en ejecutarse el método: 0.06413227319717407 +- 0.005516524172943507s
Tiempo en ejecutarse la búsqueda 6.35749363899231s, (0.10595822731653849 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVMBag:
    Best params -> {'bootstrap': True, 'estimator__C': 0.01, 'estimator__gamma': 0.01, 'estimator__kernel': 'rbf', 'max_features': 0.5, 'n_estimators': 10}
    Best score -> 0.6614942528735632
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.6197916666666666
Tiempo medio en ejecutarse el método: 0.2152583345770836 +- 0.04198128183705442s
Tiempo en ejecu

In [30]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBaggingTree = []
for i in data:
    init = time.time()
    optimalTreeBag.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBag:")
    print(f'    Best params -> {optimalTreeBag.best_params_}')
    print(f'    Best score -> {optimalTreeBag.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBag.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBag.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBag.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBag.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBaggingTree.append(optimalTreeBag.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBaggingTree)
TreeBagDF = pd.DataFrame (my_dict)
print(TreeBagDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBag:
    Best params -> {'bootstrap': False, 'estimator__criterion': 'gini', 'estimator__splitter': 'random', 'max_features': 0.75}
    Best score -> 0.9357549857549857
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9659090909090909
Tiempo medio en ejecutarse el método: 0.05579126278559367 +- 0.007111126282347554s
Tiempo en ejecutarse la búsqueda 11.740135431289673s, (0.19566892385482787 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBag:
    Best params -> {'bootstrap': True, 'estimator__criterion': 'entropy', 'estimator__splitter': 'random', 'max_features': 0.75}
    Best score -> 0.7673018753781004
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7083333333333334
Tiempo medio en ejecutarse el método: 0.051435379187266046 +- 0.0064023410216155s
Tiempo en ejecutarse la búsqueda 10.93658995628357s, (0.182276499

In [31]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting1_SVM = []
for i in data:
    init = time.time()
    optimalSVMBoost1.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost1:")
    print(f'    Best params -> {optimalSVMBoost1.best_params_}')
    print(f'    Best score -> {optimalSVMBoost1.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBoost1.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost1.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBoost1.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBoost1.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting1_SVM.append(optimalSVMBoost1.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting1_SVM)
SVMBoost1DF = pd.DataFrame (my_dict)
print(SVMBoost1DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVMBoost1:
    Best params -> {'estimator__C': 100, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'learning_rate': 0.1, 'n_estimators': 25}
    Best score -> 0.947008547008547
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9886363636363636
Tiempo medio en ejecutarse el método: 0.4636915627453062 +- 0.07680047199395146s
Tiempo en ejecutarse la búsqueda 181.99045038223267s, (3.033174173037211 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVMBoost1:
    Best params -> {'estimator__C': 1000, 'estimator__gamma': 0.01, 'estimator__kernel': 'rbf', 'learning_rate': 1, 'n_estimators': 50}
    Best score -> 0.668421052631579
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.6041666666666666
Tiempo medio en ejecutarse el método: 1.6354723155498503 +- 0.4704846335806997s
Tiempo en ejecutarse la búsqueda 633.9929716587067s, 

In [32]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting1_Tree = []
for i in data:
    init = time.time()
    optimalTreeBoost1.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBoost1:")
    print(f'    Best params -> {optimalTreeBoost1.best_params_}')
    print(f'    Best score -> {optimalTreeBoost1.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBoost1.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBoost1.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBoost1.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBoost1.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting1_Tree.append(optimalTreeBoost1.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting1_Tree)
TreeBoost1DF = pd.DataFrame (my_dict)
print(TreeBoost1DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost1:
    Best params -> {'estimator__criterion': 'log_loss', 'estimator__splitter': 'random', 'learning_rate': 10, 'n_estimators': 25}
    Best score -> 0.9085470085470085
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9204545454545454
Tiempo medio en ejecutarse el método: 0.010336774587631223 +- 0.0017734523519416026s
Tiempo en ejecutarse la búsqueda 2.8045172691345215s, (0.04674195448557536 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost1:
    Best params -> {'estimator__criterion': 'gini', 'estimator__splitter': 'best', 'learning_rate': 10, 'n_estimators': 25}
    Best score -> 0.7275559588626741
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.6666666666666666
Tiempo medio en ejecutarse el método: 0.010437184572219848 +- 0.002403138391451731s
Tiempo en ejecutarse la búsqueda 2.887934923171997s, (0.0

In [33]:
"""
Datasets = []
TimeSearch = []
TimeMethod = []
Score = []
for i in data:
    init = time.time()
    optimalSVMBoost2.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost2:")
    print(f'    Best params -> {optimalSVMBoost2.best_params_}')
    print(f'    Best score -> {optimalSVMBoost2.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBoost2.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost2.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBoost2.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBoost2.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    Score.append(optimalSVMBoost2.score(i[3], i[4]))

d7 = dict(SVM_Boost2=Score)
my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=Score)
SVMBoost2DF = pd.DataFrame (my_dict)
print(SVMBoost2DF.to_latex())
"""

'\nDatasets = []\nTimeSearch = []\nTimeMethod = []\nScore = []\nfor i in data:\n    init = time.time()\n    optimalSVMBoost2.fit(i[1], i[2])\n    end = time.time()\n    timeSearch = end - init\n    print(f"\n--------- {i[0]} ---------")\n    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost2:")\n    print(f\'    Best params -> {optimalSVMBoost2.best_params_}\')\n    print(f\'    Best score -> {optimalSVMBoost2.best_score_}\')\n\n    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")\n    print(f"    score = {optimalSVMBoost2.score(i[3], i[4])}")\n    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost2.cv_results_.get(\'mean_fit_time\'))} +- {np.mean(optimalSVMBoost2.cv_results_.get(\'std_fit_time\'))}s")\n    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")\n    Datasets.append(i[0])\n    TimeMethod.append(np.mean(optimalSVMBoost2.cv_results_.get(\'mean_fit_time\')))\n    TimeSearch.append(timeSearch)

In [34]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting2_Tree = []
for i in data:
    init = time.time()
    optimalTreeBoost2.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBoost2:")
    print(f'    Best params -> {optimalTreeBoost2.best_params_}')
    print(f'    Best score -> {optimalTreeBoost2.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBoost2.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBoost2.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBoost2.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBoost2.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting2_Tree.append(optimalTreeBoost2.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting2_Tree)
TreeBoost2DF = pd.DataFrame (my_dict)
print(TreeBoost2DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost2:
    Best params -> {'estimator__criterion': 'entropy', 'estimator__splitter': 'random', 'learning_rate': 1, 'n_estimators': 25}
    Best score -> 0.9163817663817664
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9318181818181818
Tiempo medio en ejecutarse el método: 0.010791830221811929 +- 0.0016796092941906557s
Tiempo en ejecutarse la búsqueda 3.0393548011779785s, (0.050655913352966306 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost2:
    Best params -> {'estimator__criterion': 'gini', 'estimator__splitter': 'best', 'learning_rate': 10, 'n_estimators': 25}
    Best score -> 0.7329098608590441
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.703125
Tiempo medio en ejecutarse el método: 0.012744296259350248 +- 0.00344316060557427s
Tiempo en ejecutarse la búsqueda 3.578855514526367s, (0.0596475919087

In [35]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreGradBoost = []
for i in data:
    init = time.time()
    optimalGradBoost.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente GradBoost:")
    print(f'    Best params -> {optimalGradBoost.best_params_}')
    print(f'    Best score -> {optimalGradBoost.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalGradBoost.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalGradBoost.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalGradBoost.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalGradBoost.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreGradBoost.append(optimalGradBoost.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreGradBoost)
GradBoostDF = pd.DataFrame (my_dict)
print(GradBoostDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'squared_error', 'learning_rate': 1, 'loss': 'exponential', 'n_estimators': 100, 'warm_start': True}
    Best score -> 0.9279202279202279
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9545454545454546
Tiempo medio en ejecutarse el método: 0.38358656962712606 +- 0.04183697571233041s
Tiempo en ejecutarse la búsqueda 71.23071599006653s, (1.1871785998344422 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 100, 'warm_start': True}
    Best score -> 0.7638838475499092
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7083333333333334
Tiempo medio en ejecutarse el método: 0.2663808359040154 +- 0.02545372676082246s
Tiempo en ejecutarse la búsqueda 49.71514868736267s,

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 444, in fit
    self._check_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 301, in _check_params
    self._loss = loss_class(self.n_classes_)
  File "/usr/local/lib/python3.10/dist-packages/skl


--------- Iris ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 100, 'warm_start': True}
    Best score -> 0.9734848484848484
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.8947368421052632
Tiempo medio en ejecutarse el método: 0.2828403274218242 +- 0.030977160311430236s
Tiempo en ejecutarse la búsqueda 52.46371912956238s, (0.8743953188260396 min)
\begin{tabular}{llrrr}
\toprule
{} &     Dataset &  TimeMethod &  TimeSearch &     Score \\
\midrule
0 &  Ionosphere &    0.383587 &   71.230716 &  0.954545 \\
1 &    Diabetes &    0.266381 &   49.715149 &  0.708333 \\
2 &     Vehicle &    0.387424 &   71.825253 &  0.952830 \\
3 &       Vowel &    0.454113 &   83.768230 &  1.000000 \\
4 &        Iris &    0.282840 &   52.463719 &  0.894737 \\
\bottomrule
\end{tabular}



In [36]:
compareDF = pd.DataFrame()
compareDF['SVM'] = ScoreSVM
compareDF['Tree'] = ScoreTree
compareDF['BaggingSVM'] = ScoreBaggingSVM
compareDF['BaggingTree'] = ScoreBaggingTree
compareDF['Boosting1_SVM'] = ScoreBoosting1_SVM
compareDF['Boosting1_Tree'] = ScoreBoosting1_Tree
compareDF['Boosting2_Tree'] = ScoreBoosting2_Tree
compareDF['GradBoost'] = ScoreGradBoost
compareDF.to_csv("performanceGridSearch.csv", index=False)
print(compareDF)
print(compareDF.to_latex())

        SVM      Tree  BaggingSVM  BaggingTree  Boosting1_SVM  Boosting1_Tree  \
0  0.875000  0.931818    0.840909     0.965909       0.988636        0.920455   
1  0.750000  0.656250    0.619792     0.708333       0.604167        0.666667   
2  0.966981  0.957547    0.764151     0.976415       0.792453        0.962264   
3  0.963563  0.987854    0.919028     0.995951       1.000000        0.987854   
4  0.921053  0.973684    0.815789     0.894737       0.921053        0.894737   

   Boosting2_Tree  GradBoost  
0        0.931818   0.954545  
1        0.703125   0.708333  
2        0.943396   0.952830  
3        0.975709   1.000000  
4        0.947368   0.894737  
\begin{tabular}{lrrrrrrrr}
\toprule
{} &       SVM &      Tree &  BaggingSVM &  BaggingTree &  Boosting1\_SVM &  Boosting1\_Tree &  Boosting2\_Tree &  GradBoost \\
\midrule
0 &  0.875000 &  0.931818 &    0.840909 &     0.965909 &       0.988636 &        0.920455 &        0.931818 &   0.954545 \\
1 &  0.750000 &  0.656250 &   

# Iman Davenport test

In [37]:
import numpy as np
from scipy.stats import chi2

def compute_imon_davenport_statistic(performance_matrix):
  # Compute the ranks of the model performance on each dataset
  ranks = np.apply_along_axis(lambda x: len(x) - np.argsort(np.argsort(x)), 1, performance_matrix)

  # Sum the ranks for each model across all datasets
  rank_sums = np.sum(ranks, axis=0)

  # Compute the iman Davenport statistic
  iman_davenport_statistic = (np.max(rank_sums) - np.min(rank_sums)) / performance_matrix.shape[1]

  return iman_davenport_statistic

def compute_p_value(imon_davenport_statistic, num_models, num_datasets):
  # Compute the degrees of freedom for the Imon Davenport test
  df = num_models - 1

  # Compute the p-value using the chi-squared distribution
  p_value = 1 - chi2.cdf(imon_davenport_statistic, df)

  return p_value

def imon_davenport_test(performance_matrix, significance_level):
  # Compute the Imon Davenport statistic and p-value
  imon_davenport_statistic = compute_imon_davenport_statistic(performance_matrix)
  p_value = compute_p_value(imon_davenport_statistic, performance_matrix.shape[0], performance_matrix.shape[1])

  # Determine whether the difference in performance between the models is statistically significant
  if p_value < significance_level:
    print(f"The difference in performance between the models is statistically significant (p = {p_value:.3f})")
  else:
    print(f"The difference in performance between the models is not statistically significant (p = {p_value:.3f})")
  return p_value

In [38]:
performance_matrix = pd.read_csv("performanceGridSearch.csv")
# Run the Imon Davenport test
p_value = imon_davenport_test(performance_matrix, 0.05)

The difference in performance between the models is not statistically significant (p = 0.579)


# Wilcoxon Test

In [39]:
# If there are significant differences between the models, then apply the wilcoxon test to determine which models are significantly different
if p_value < 0.05:
  # Compute the pairwise differences between the models
  pairwise_differences = np.apply_along_axis(lambda x: x - x[:, None], 1, performance_matrix)

  # Compute the p-values for the pairwise differences using the Wilcoxon signed-rank test
  p_values = np.apply_along_axis(lambda x: wilcoxon(x, zero_method="wilcox")[1], 1, pairwise_differences)

  # Compute the Bonferroni correction
  bonferroni_correction = 0.05 / (p_values.shape[0] * (p_values.shape[0] - 1) / 2)

  # Determine which models are significantly different
  significant_differences = np.argwhere(p_values < bonferroni_correction)

  # Print the significant differences
  for difference in significant_differences:
    print(f"Model {difference[0]} is significantly different from model {difference[1]}")